In [1]:
import pandas as pd
import warnings

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_parquet('../data/yellow_tripdata_2023-01.parquet')
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [3]:
df.shape

(3066766, 19)

>> Q1: 19

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[us]
 2   tpep_dropoff_datetime  datetime64[us]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

In [5]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.dt.total_seconds() / 60
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00,8.433333
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00,6.316667
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00,12.750000
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25,9.616667
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00,10.833333


In [6]:
df.duration.std()

np.float64(42.59435124195458)

>> Q2: 42.59

In [7]:
df.describe()['duration']

count    3.066766e+06
mean     1.566900e+01
min     -2.920000e+01
25%      7.116667e+00
50%      1.151667e+01
75%      1.830000e+01
max      1.002918e+04
std      4.259435e+01
Name: duration, dtype: float64

In [8]:
df.shape

(3066766, 20)

In [9]:
df = df[(df.duration >= 1) & (df.duration <= 60)]
df.shape

(3009173, 20)

In [10]:
3009173 / 3066766

0.9812202822125979

>> Q3: 98%

In [11]:
train_df = df[['PULocationID', 'DOLocationID']]
train_df['PULocationID'] = train_df['PULocationID'].astype(str)
train_df['DOLocationID'] = train_df['DOLocationID'].astype(str)
train_dict = train_df.to_dict(orient='records')

In [12]:
dict_vec = DictVectorizer(sparse=False)

X = dict_vec.fit_transform(train_dict)

In [13]:
X.shape

(3009173, 515)

>> Q4: 515

In [14]:
y = df['duration'].values

lr = LinearRegression().fit(X, y)

y_pred = lr.predict(X)
root_mean_squared_error(y, y_pred)

7.649261024900074

>>Q5: 7.64

In [15]:
validation_df = pd.read_parquet('../data/yellow_tripdata_2023-02.parquet')
validation_df['duration'] = validation_df.tpep_dropoff_datetime - validation_df.tpep_pickup_datetime
validation_df.duration = validation_df.duration.dt.total_seconds() / 60
validation_df = validation_df[(validation_df.duration >= 1) & (validation_df.duration <= 60)].copy()
validation = validation_df[['PULocationID', 'DOLocationID']]
validation['PULocationID'] = validation['PULocationID'].astype(str)
validation['DOLocationID'] = validation['DOLocationID'].astype(str)
val_dict = validation.to_dict(orient='records')

In [16]:
val = dict_vec.transform(val_dict)
y_val = lr.predict(val)
y_true = validation_df['duration'].values
root_mean_squared_error(y_true, y_val)

7.811832798910325

>>Q6: 7.81